# Session 10: Using Ragas to Evaluate a RAG Application built with LangChain and LangGraph

In the following notebook, we'll be looking at how [Ragas](https://github.com/explodinggradients/ragas) can be helpful in a number of ways when looking to evaluate your RAG applications!

While this example is rooted in LangChain/LangGraph - Ragas is framework agnostic (you don't even need to be using a framework!).

## 🤝 Breakout Room #1
  - Task 1: Installing Required Libraries
  - Task 2: Set Environment Variables
  - Task 3: Synthetic Dataset Generation for Evaluation using Ragas
  - Task 4: Construct our RAG application
  - Task 5: Evaluating our Application with Ragas
  - Task 6: Making Adjustments and Re-Evaluating
  - ***Activity #1: Implement a Different Reranking Strategy***


## Task 1: Installing Required Libraries

If you have not already done so, install the required libraries using the uv package manager:
``` bash

uv sync

```


## Task 2: Set Environment Variables:

We'll also need to provide our API keys.
> NOTE: In addition to OpenAI's models, this notebook will be using Cohere's Reranker - please be sure to [sign-up for an API key!](https://docs.cohere.com/reference/about)

You have two options for supplying your API keys in this session:
- Use environment variables (see Prerequisite #2 in the README.md)
- Provide them via a prompt when the notebook runs

The following code will load all of the environment variables in your `.env`. Then, it checks for the two API keys we need. If they are not there, it will prompt you to provide them.

First, OpenAI's for our LLM/embedding model combination!

Second, Cohere's for our reranking


In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key!")

if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass("Please enter your Cohere API key!")

## Task 3: Synthetic Dataset Generation for Evaluation using Ragas

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data using the Health & Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, and stress management.

Next, let's load our data into a familiar LangChain format using the `TextLoader`.

In [5]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/HealthWellnessGuide.txt")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [7]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/4 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/9 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,As someone who is always looking for practical...,[The Personal Wellness Guide A Comprehensive R...,The Bird Dog exercise is recommended for lower...,single_hop_specifc_query_synthesizer
1,what i do for lower back pain? exercises help ...,[The Personal Wellness Guide A Comprehensive R...,Lower back pain affects about 80% of adults so...,single_hop_specifc_query_synthesizer
2,How can the Knee-to-Chest Stretch be incorpora...,[The Personal Wellness Guide A Comprehensive R...,The Knee-to-Chest Stretch can be included in a...,single_hop_specifc_query_synthesizer
3,"What is CBT-I, and how can it help manage inso...",[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,"CBT-I, or Cognitive Behavioral Therapy for Ins...",single_hop_specifc_query_synthesizer
4,Wut are sum benifits of chamomile tee for slee...,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,"According to the context, herbal teas such as ...",single_hop_specifc_query_synthesizer
5,magnesium help sleep?,[PART 3: SLEEP AND RECOVERY Chapter 7: The Sci...,Magnesium supplements (consult healthcare prov...,single_hop_specifc_query_synthesizer
6,what chapter 19 say about work life balance? i...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 19 say maintaining balance between pro...,single_hop_specifc_query_synthesizer
7,Wut are the main stategies for boostin immune ...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,Chapter 18 recomends several immune-boosting s...,single_hop_specifc_query_synthesizer
8,Wut are sum key tips from PART 5 for bilding h...,[PART 5: BUILDING HEALTHY HABITS Chapter 13: T...,PART 5 recomends starting small (like commitin...,single_hop_specifc_query_synthesizer


## Task 4: Construct our RAG application

Now we'll construct our LangChain RAG, which we will be evaluating using the above created test data!

### R - Retrieval

Let's start with building our retrieval pipeline, which will involve loading the same data we used to create our synthetic test set above.

> NOTE: We need to use the same data - as our test set is specifically designed for this data.

In [9]:
loader = TextLoader("data/HealthWellnessGuide.txt")
docs = loader.load()

Now that we have our data loaded, let's split it into chunks!

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=0)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

447

### ❓ Question #1:

What is the purpose of the `chunk_overlap` parameter in the `RecursiveCharacterTextSplitter`?

##### Answer:

*Your answer here*

Next up, we'll need to provide an embedding model that we can use to construct our vector store.

In [11]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can build our in memory QDrant vector store.

In [12]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="use_case_data",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="use_case_data",
    embedding=embeddings,
)

We can now add our documents to our vector store.

In [13]:
_ = vector_store.add_documents(documents=split_documents)

Let's define our retriever.

In [14]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

Now we can produce a node for retrieval!

In [15]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

### A - Augmented

Let's create a simple RAG prompt!

In [16]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context, and cannot use your own knowledge.

### Question
{question}

### Context
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### G - Generation

We'll also need an LLM to generate responses - we'll use `gpt-4o-nano` to avoid using the same model as our judge model.

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")

Then we can create a `generate` node!

In [18]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = llm.invoke(messages)
  return {"response" : response.content}

### Building RAG Graph with LangGraph

Let's create some state for our LangGraph RAG graph!

In [19]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

Now we can build our simple graph!

> NOTE: We're using `add_sequence` since we will always move from retrieval to generation. This is essentially building a chain in LangGraph.

In [20]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

Let's do a test to make sure it's doing what we'd expect.

In [21]:
response = graph.invoke({"question" : "What exercises help with lower back pain?"})

In [22]:
response["response"]

'The provided context does not specify which exercises help with lower back pain.'

## Task 5: Evaluating our Application with Ragas

Now we can finally do our evaluation!

We'll start by running the queries we generated usign SDG above through our application to get context and responses.

In [23]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]

In [24]:
dataset.samples[0].eval_sample.response

'The Bird Dog exercise is performed starting on your hands and knees. To do it properly, extend one arm forward while simultaneously extending the opposite leg backward. Keep your back flat and engaged, ensuring your hips and shoulders stay level. Hold this position briefly, then return to the starting position and repeat with the other arm and leg.\n\nThis exercise helps strengthen the core muscles, including the lower back and abdomen, which support the spine. By improving core stability and promoting proper spinal alignment, the Bird Dog can be an effective and practical way to relieve lower back pain.'

Then we can convert that table into a `EvaluationDataset` which will make the process of evaluation smoother.

In [25]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

We'll need to select a judge model - in this case we're using the same model that was used to generate our Synthetic Data.

In [26]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))

Next up - we simply evaluate on our desired metrics!

In [27]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=360)

baseline_result = evaluate(
    dataset=evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
baseline_result

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

{'context_recall': 0.2963, 'faithfulness': 0.6595, 'factual_correctness': 0.3933, 'answer_relevancy': 0.5172, 'context_entity_recall': 0.3280, 'noise_sensitivity_relevant': 0.0000}

## Task 6: Making Adjustments and Re-Evaluating

Now that we've got our baseline - let's make a change and see how the model improves or doesn't improve!




We'll first set our retriever to return more documents, which will allow us to take advantage of the reranking.

In [29]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
split_documents = text_splitter.split_documents(docs)
len(split_documents)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="use_case_data_new_chunks",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="use_case_data_new_chunks",
    embedding=embeddings,
)

_ = vector_store.add_documents(documents=split_documents)

adjusted_example_retriever = vector_store.as_retriever(search_kwargs={"k": 20})

Reranking, or contextual compression, is a technique that uses a reranker to compress the retrieved documents into a smaller set of documents.

This is essentially a slower, more accurate form of semantic similarity that we use on a smaller subset of our documents.

In [30]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

def retrieve_adjusted(state):
  compressor = CohereRerank(model="rerank-v3.5")
  compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=adjusted_example_retriever, search_kwargs={"k": 5}
  )
  retrieved_docs = compression_retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

We can simply rebuild our graph with the new retriever!

In [31]:
class AdjustedState(TypedDict):
  question: str
  context: List[Document]
  response: str

adjusted_graph_builder = StateGraph(AdjustedState).add_sequence([retrieve_adjusted, generate])
adjusted_graph_builder.add_edge(START, "retrieve_adjusted")
adjusted_graph = adjusted_graph_builder.compile()

In [32]:
response = adjusted_graph.invoke({"question" : "How can I improve my sleep quality?"})
response["response"]

'To improve your sleep quality, consider practicing good sleep hygiene by maintaining a consistent sleep schedule, creating a relaxing bedtime routine, and keeping your bedroom cool, dark, and quiet. Limit screen exposure 1-2 hours before bed and avoid caffeine after 2 PM. Regular exercise (but not close to bedtime) and limiting alcohol and heavy meals before sleep can also help. Use tools like blackout curtains or sleep masks, choose a comfortable mattress and pillows, and aim for a consistent wake-up time. Additionally, relaxing techniques such as progressive muscle relaxation, herbal teas like chamomile or valerian root, meditation, and deep breathing exercises can promote better sleep. If necessary, cognitive behavioral therapy for insomnia (CBT-I) and consulting a healthcare provider about magnesium supplements may also be beneficial.'

In [33]:
import time
import copy

rerank_dataset = copy.deepcopy(dataset)

for test_row in rerank_dataset:
  response = adjusted_graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]
  time.sleep(2) # To try to avoid rate limiting.

In [34]:
rerank_dataset.samples[0].eval_sample.response

'To properly perform the Bird Dog exercise, follow these steps:\n\n1. Start on your hands and knees on a comfortable, flat surface.\n2. Engage your core muscles to maintain stability throughout the movement.\n3. Simultaneously extend your opposite arm and leg — for example, extend your right arm forward and your left leg backward.\n4. Keep your back flat and avoid arching or sagging.\n5. Hold this extended position for about 5 seconds, focusing on maintaining good balance and core engagement.\n6. Return your hand and knee to the starting position.\n7. Repeat the movement on the opposite side, extending your left arm and right leg.\n8. Do 10 repetitions per side, ensuring controlled and steady movements.\n\n**Benefits for Lower Back Pain Relief:**\nThe Bird Dog exercise helps strengthen the core muscles, including the lower back and abdominals, which are essential for supporting your spine. By improving core stability, it reduces unnecessary stress on the lower back, alleviating discomf

In [35]:
rerank_evaluation_dataset = EvaluationDataset.from_pandas(rerank_dataset.to_pandas())

In [36]:
rerank_result = evaluate(
    dataset=rerank_evaluation_dataset,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
rerank_result

Evaluating:   0%|          | 0/54 [00:00<?, ?it/s]

{'context_recall': 0.9630, 'faithfulness': 0.7518, 'factual_correctness': 0.7267, 'answer_relevancy': 0.9521, 'context_entity_recall': 0.4537, 'noise_sensitivity_relevant': 0.0171}

### ❓ Question #2:

Which system performed better, on what metrics, and why?

##### Answer:

*Your answer here*

### ❓ Question #3:

What are the benefits and limitations of using synthetic data generation for RAG evaluation? Consider both the practical advantages and potential pitfalls.

##### Answer:

*Your answer here*

### ❓ Question #4:

If you were building a production wellness assistant, which Ragas metrics would be most important to optimize for and why? Consider the healthcare/wellness domain specifically.

##### Answer:

*Your answer here*

## Activity #1: Implement a Different Reranking Strategy

In this activity, you'll experiment with different reranking parameters or strategies to see how they affect the evaluation metrics.

**Requirements:**
1. Modify the `retrieve_adjusted` function to use different parameters (e.g., change `k` values, try different top_n for reranking)
2. Or implement a different retrieval enhancement strategy (e.g., hybrid search, query expansion)
3. Run the evaluation and compare results with the baseline and reranking results above
4. Document your findings in the markdown cell below

In [ ]:
### YOUR CODE HERE ###

# Implement your custom retrieval strategy here
# Example: modify retrieve_adjusted with different parameters

def retrieve_custom(state):
    # Your implementation here
    pass

### Activity #1 Findings:

*Document your findings here: What strategy did you try? How did it compare to the baseline and reranking results?*